In [1]:
from sys import stdout
from os.path import join, isdir, isfile, exists
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import scipy.integrate as integrate
from random import sample
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import time

In [2]:
washing_machines = pd.read_hdf('./washing_machines.h5')
#washing_machines = pd.read_hdf('./dish_washers.h5')
#washing_machines = pd.read_hdf('./tumble_dryers.h5')
aggregates = pd.read_hdf('./aggregate.h5')

In [3]:
# ADD TO LIB
def getMinMaxDates(df):
    # find the dates that exist for all measurements
    maxdate = df.index.get_level_values('Unix').max()
    mindate = df.index.get_level_values('Unix').min()
    # for all buildings get the common denominator for min and max dates
    for i in np.unique(df.index.get_level_values('building')):
        mask = df.index.get_level_values('building') == i
        if (df.loc[mask].index.get_level_values('Unix').max()) < maxdate:
            maxdate = df.loc[mask].index.get_level_values('Unix').max()
        if (df.loc[mask].index.get_level_values('Unix').min()) > mindate:
            mindate = df.loc[mask].index.get_level_values('Unix').min()
    return (mindate, maxdate)

(mindate,maxdate) = getMinMaxDates(washing_machines)
print(mindate, maxdate)

2014-06-03 11:30:00 2015-05-10 23:30:00


In [4]:
timedelta = maxdate-mindate
trainset = mindate + timedelta/2

In [6]:
trainset

Timestamp('2014-11-21 05:30:00')

In [ ]:
from datetime import date, timedelta

sdate = date(trainset.year, trainset.month, trainset.day)   # start date
edate = date(maxdate.year, maxdate.month, maxdate.day)   # end date

delta = edate - sdate       # as timedelta

dates = []

for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    dates.append(day)

print(len(dates))
sum(states[2::20])
from itertools import compress
list(compress(dates,states[2::20]))

In [7]:
def on_times(appliance):
    """
        Parameters
        ----------
        appliance = a dataframe of an appliance witn an 'active' column

        Returns
        A dataframe with a datetime index and a column duration.
        Index is when the appliance started and duration the amount of slots the appliance was on (power>20watt).

    """
    records = len(appliance)
    on_dur = pd.DataFrame(columns=['duration'])
    i = 0
    # TODO(kyrcha): possible optimization with matrix
    while (i < records):
        j = 0
        if appliance['active'].values[i] > 50:
            # to parakatw to evala giati evgaze proeidopoihsh gia nan pou nomizw den isxye kiolas
            # todo: check if nan exist.
            np.warnings.filterwarnings('ignore')
            #αν δεν είναι το τελευταιο στοιχείο του πίνακα γιατί πετούσε έρρορ
            while appliance['active'].values[i + j] > 50:
                if j == 0:
                    on_time = appliance.index.get_level_values('Unix')[i]
                j = j + 1
                if ((i + j) == records):
                    break
            on_dur.loc[on_time] = j
            i = i + j -1
        i = i + 1
    return on_dur

In [94]:
# playground for creating the yesterday component

df = washing_machines
mask = df.index.get_level_values('building') == 21
idx = pd.IndexSlice
new_app = df.loc[idx[mask, :, mindate:trainset]]
on_app = on_times(new_app) # this returns the start datetime and how many timeslots the appliance stayed open
# for loop
# probabiliy of on given yesterday...
today_and_yesterday = [opened_today_and_yesterday(x, on_app.index.date) for x in new_app.index.get_level_values('Unix').date]
only_yesterday = [opened_yesterday_but_not_today(x, on_app.index.date) for x in new_app.index.get_level_values('Unix').date]
only_today = [opened_today_but_not_yesterday(x, on_app.index.date) for x in new_app.index.get_level_values('Unix').date]
neither = [neither_today_nor_yesterday(x, on_app.index.date) for x in new_app.index.get_level_values('Unix').date]

In [116]:
# Get on dates for all buildings
on_times_all = []
for i in np.unique(df.index.get_level_values('building')):
    mask = df.index.get_level_values('building') == i
    idx = pd.IndexSlice
    new_app = df.loc[idx[mask, :, :]]
    on_times_all.append(on_times(new_app).index.date)
on_times_all

[array([datetime.date(2013, 10, 23), datetime.date(2013, 10, 23),
        datetime.date(2013, 10, 23), datetime.date(2013, 10, 24),
        datetime.date(2013, 10, 26), datetime.date(2013, 10, 27),
        datetime.date(2013, 10, 28), datetime.date(2013, 10, 29),
        datetime.date(2013, 10, 29), datetime.date(2013, 10, 30),
        datetime.date(2013, 10, 31), datetime.date(2013, 11, 2),
        datetime.date(2013, 11, 3), datetime.date(2013, 11, 4),
        datetime.date(2013, 11, 7), datetime.date(2013, 11, 10),
        datetime.date(2013, 11, 12), datetime.date(2013, 11, 12),
        datetime.date(2013, 11, 13), datetime.date(2013, 11, 14),
        datetime.date(2013, 11, 15), datetime.date(2013, 11, 19),
        datetime.date(2013, 11, 19), datetime.date(2013, 11, 22),
        datetime.date(2013, 11, 22), datetime.date(2013, 11, 23),
        datetime.date(2013, 11, 25), datetime.date(2013, 11, 26),
        datetime.date(2013, 11, 27), datetime.date(2013, 11, 27),
        dateti

In [121]:
len(on_times_all)

20

In [95]:
print(sum(today_and_yesterday))
print(len(today_and_yesterday))
print(sum(only_yesterday))
print(len(only_yesterday))
print(sum(only_today))
print(len(only_today))
print(sum(neither))
print(len(neither))
np.unique(df.index.get_level_values('building'))

2496
16393
2976
16393
2976
16393
7945
16393


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 17, 18,
       19, 20, 21])

In [107]:
def prob_on_given_yesterday(appliance):
    '''

    Parameters

    appliance

    Returns
    probs = a list of probabilities for each day

    '''
    new_app = appliance
    on_app = on_times(new_app)

    if on_app.empty:
        print('The appliance was never on')
        probs = []
    else:
        today_and_yesterday = [opened_today_and_yesterday(x, on_app.index.date) for x in new_app.index.get_level_values('Unix').date]
        #only_yesterday = [opened_yesterday_but_not_today(x, on_app.index.date) for x in new_app.index.get_level_values('Unix').date]
        only_today = [opened_today_but_not_yesterday(x, on_app.index.date) for x in new_app.index.get_level_values('Unix').date]
        #neither = [neither_today_nor_yesterday(x, on_app.index.date) for x in new_app.index.get_level_values('Unix').date]
        sum_today_and_yesterday = sum(today_and_yesterday)
        sum_only_today = sum(only_today)
        total = sum_today_and_yesterday + sum_only_today
        probs = []
        probs.append(sum_only_today/total)
        probs.append(sum_today_and_yesterday/total)
    return probs

In [97]:
def prob_time_new(appliance):
    '''

    Parameters
    ----------
    appliance = a dataframe of an appliance witn an 'active' column

    Returns
    a series of diffent start times & how many times they occured
    an array of the same thing

    '''
    #fig = plt.figure()
    #ax = fig.add_subplot(1,1,1)
    #fig.suptitle(name + ' time on', fontsize=16)
    new_app = appliance
    # check  mer
    if new_app.empty:
        print('The appliance was never on')
        ptonosx = []
    else:
        on_app = on_times(new_app)
        if len(on_app) > 1:
            #κάνω μια στηλη μόνο με τις ώρες από το Datatime index
            on_app['time'] = on_app.index.time
            #μου εβγαζε error αν εβαζα κατευθείαν το index στο GMM και γ αυτό έκανα όλο αυτό με τους πίνακες ώστε να φτάσω στο W
            Y = on_app['time'].index.to_list()
            X = []
            for i in range(0, len(Y)):
               X.append(Y[i].hour * 3600 + Y[i].minute * 60 + Y[i].second)
            fullX = X.copy()

            # need to plot these here...
            # do everything with  1 house in mind and plot

            # φτιάχνουμε ένα νέο population όπου αντί για 0-24h, θα απλώνεται σε -6h ως 30h,
            for i in range(0, len(fullX)):
                # προσθέτοντας τα samples που είναι στις ώρες 0-6 στις ώρες 24-30
                if (fullX[i] < 21600):
                    fullX.append(86400 + fullX[i])
                # προσθέτοντας τα samples που είναι στις ώρες 0-6 στις ώρες 24-30
                elif (fullX[i] > 64800):
                    fullX.append(fullX[i] - 86400)
            # μετακινούμε αυτό το νέο population σε 0-36h
            for i in range(0, len(fullX)):
                fullX[i] = fullX[i] + 21600
            fullXX = np.array(fullX)
            fullXX = fullXX.reshape(-1, 1)
            if fullXX.size < 10:
                # return empty array if data are not enough
                return []
            gmm = GaussianMixture(n_components=10, covariance_type='diag', reg_covar=100, n_init=5).fit(fullXX)
            x = np.arange(0, 129600, 1)
            x = x.reshape(-1, 1)
            p = np.exp(gmm.score_samples(x))
            # κρατάμε τις πιθανότητες από τις ώρες που μας ενδιαφέρουν όντως και κανονικοποιούμε
            ptonosx = p[21600:108000] / p[21600:108000].sum()

            #ax.plot(y,probability)
            #ax.hist(fullXX,bins=96,density=True)
            #ax.set_title(str(len(Y)) + ' times it was on', fontsize=12)
            #plt.show()
        else:
            print('The appliance was on 1 time')
            ptonosx = []
    return (ptonosx)

In [98]:
def prob_duration(appliance):
    """

    Parameters
    ----------
    appliance = a dataframe of an appliance
    name = the name of the appliance
    Returns
    a

    """
    idx = pd.IndexSlice
    new_app = appliance
    on_app = on_times(new_app)
    probs = on_app['duration'].value_counts(normalize=True).sort_index().to_frame()
    probs = probs.loc[idx[probs.index<96]] # keep one day's worth of duration...else next day
    #fig = plt.figure()
    #fig.suptitle(name + ' duration', fontsize=16)
    #ax = fig.add_subplot(1, 1, 1)
    #ax.bar(probs.index, probs)
    return probs

In [122]:
from datetime import datetime, timedelta

def opened_today_and_yesterday(today, array):
    yesterday = today - timedelta(days=1)
    return (today in array and yesterday in array)

def opened_yesterday_but_not_today(today, array):
    yesterday = today - timedelta(days=1)
    return (today not in array and yesterday in array)

def opened_today_but_not_yesterday(today, array):
    yesterday = today - timedelta(days=1)
    return (today in array and yesterday not in array)

def neither_today_nor_yesterday(today, array):
    yesterday = today - timedelta(days=1)
    return (today not in array and yesterday not in array)

def off_yesterday(date, array):
    yesterday = date - timedelta(days=1)
    return (yesterday in array)*1

In [29]:
df = washing_machines
mask = df.index.get_level_values('building') == 1
idx = pd.IndexSlice
new_app = df.loc[idx[mask, :, mindate:trainset]]
[is_business_day(x) for x in new_app.index.get_level_values('Unix')]
on_app = on_times(new_app)

#

[(0, Timestamp('2014-06-08 11:45:00')),
 (0, Timestamp('2014-06-08 13:15:00')),
 (1, Timestamp('2014-06-11 12:45:00')),
 (1, Timestamp('2014-06-12 08:00:00')),
 (1, Timestamp('2014-06-12 11:00:00')),
 (1, Timestamp('2014-06-17 09:30:00')),
 (1, Timestamp('2014-06-17 11:15:00')),
 (1, Timestamp('2014-06-18 10:00:00')),
 (1, Timestamp('2014-06-19 13:45:00')),
 (1, Timestamp('2014-06-20 13:45:00')),
 (0, Timestamp('2014-06-21 14:15:00')),
 (0, Timestamp('2014-06-22 09:00:00')),
 (1, Timestamp('2014-06-25 10:15:00')),
 (1, Timestamp('2014-06-26 08:15:00')),
 (1, Timestamp('2014-06-26 10:00:00')),
 (1, Timestamp('2014-06-27 06:30:00')),
 (0, Timestamp('2014-06-29 09:15:00')),
 (1, Timestamp('2014-07-01 09:00:00')),
 (1, Timestamp('2014-07-02 12:15:00')),
 (1, Timestamp('2014-07-03 13:00:00')),
 (0, Timestamp('2014-07-06 08:45:00')),
 (1, Timestamp('2014-07-09 10:15:00')),
 (1, Timestamp('2014-07-10 08:15:00')),
 (1, Timestamp('2014-07-11 09:30:00')),
 (0, Timestamp('2014-07-12 08:15:00')),


In [123]:
# ADD TO LIB
def train(df, start , end):
    prob_times = []
    prob_days = []
    prob_dur = []
    #train the machine
    idx = pd.IndexSlice
    for i in np.unique(df.index.get_level_values('building')):
        mask = df.index.get_level_values('building') == i
        prob_days.append(prob_on_given_yesterday(df.loc[idx[mask, :, start:end]]))
        prob_times.append(prob_time_new(df.loc[idx[mask, :, start:end]]))
        prob_dur.append(prob_duration(df.loc[idx[mask, :, start:end]]))
    return prob_times, prob_days, prob_dur

prob_times, prob_days, prob_dur = train(washing_machines, mindate, trainset)

In [124]:
# check the prob days
prob_days

[[0.43157189140878116, 0.5684281085912188],
 [0.47552925108126565, 0.5244707489187344],
 [0.39051013588772554, 0.6094898641122745],
 [0.5405405405405406, 0.4594594594594595],
 [0.367816091954023, 0.632183908045977],
 [0.7878787878787878, 0.21212121212121213],
 [0.1796875, 0.8203125],
 [0.5340909090909091, 0.4659090909090909],
 [0.603448275862069, 0.39655172413793105],
 [0.20925075241565025, 0.7907492475843497],
 [0.6153846153846154, 0.38461538461538464],
 [0.25751450166988926, 0.7424854983301108],
 [0.42857142857142855, 0.5714285714285714],
 [0.75, 0.25],
 [0.5033165295834439, 0.49668347041655614],
 [0.6001074979844128, 0.3998925020155872],
 [0.7209302325581395, 0.27906976744186046],
 [0.5116279069767442, 0.4883720930232558],
 [0.7796610169491526, 0.22033898305084745],
 [0.543859649122807, 0.45614035087719296]]

In [114]:
def slot (prob_days,prob_times,prob_dur,start,end, off_yesterday):
    '''

    Parameters
    ----------
    prob_days - a list of 7 probabilities for each day
    prob_times - a list of 86400 probabilities for each second of the day
    prob_dur - probabilities of duration
    start - time of slot start
    end - time of slot end

    Returns
    -------

    '''
    idx = pd.IndexSlice
    slot_open_yesterday = off_yesterday
    start_time = start.hour * 4 + start.minute
    end_time = end.hour * 4 + end.minute
    if prob_days != [] and prob_times != []:
        the_day = prob_days[slot_open_yesterday]
        # για όταν συμπεριλάβουμε duration
        X = np.array(prob_dur.index.to_list())
        d_max = X.max()
        prob_times = prob_times.reshape(-1, 1)
        pfunctionx = lambda x: prob_times[int(x)]
        turns_on = integrate.quad(lambda x: pfunctionx(x)[0], start_time * 900, end_time * 900)[0]
        was_on = integrate.quad(lambda x: pfunctionx(start_time*900 - d_max*900 + x)[0]*prob_dur.loc[idx[prob_dur.index > (d_max-x/900)]].sum(), 0, d_max*900)[0]
        probability = the_day * (turns_on + was_on)
    else:
        probability = 0
    return probability

In [12]:
washing_machines_common = washing_machines.loc[pd.IndexSlice[:, :, trainset:maxdate], :]
mask = washing_machines_common.index.get_level_values('Unix').indexer_between_time('17:00', '19:00')
washing_machines_common_ts = washing_machines_common.iloc[mask]
washing_machines_common_ts.index = pd.MultiIndex.from_arrays([
    washing_machines_common_ts.index.get_level_values('building'), 
    washing_machines_common_ts.index.get_level_values('appNo'), 
    washing_machines_common_ts.index.get_level_values('Unix').date, 
    washing_machines_common_ts.index.get_level_values('Unix').time], 
    names=['building','appNo','date','time'])

In [13]:
aggregates_common = aggregates.loc[pd.IndexSlice[:, trainset:maxdate], :]
mask = aggregates_common.index.get_level_values('Unix').indexer_between_time('17:00', '19:00')
aggregates_common_ts = aggregates_common.iloc[mask]
aggregates_common_ts.index = pd.MultiIndex.from_arrays([
    aggregates_common_ts.index.get_level_values('building'), 
    aggregates_common_ts.index.get_level_values('Unix').date, 
    aggregates_common_ts.index.get_level_values('Unix').time], 
    names=['building', 'date','time'])

In [126]:
# for subset
subset = list(range(2,12,2))
#subset = list(range(2,4,2))
# for timeslots
# timeslots_list = [
#      ('17:00:00', '19:00:00'), 
#      ('16:00:00', '19:00:00'), 
#      ('16:00:00', '20:00:00')
#      ]
#timeslots_list = [
#    ('07:00:00', '09:00:00'), 
#    ('07:00:00', '10:00:00'), 
#    ('06:00:00', '10:00:00'),
#     ]
#timeslots_list = [

#     ('17:00:00', '19:00:00'), 
#     ('16:00:00', '19:00:00'), 
#     ('16:00:00', '20:00:00')
#     ]
timeslots_list = [('17:00:00', '19:00:00')]
# for runs
number_of_runs = 5
#number_of_runs = 1

final_grid_results = pd.DataFrame(columns=['subset', 'timeslot', 'f1_selected_score', 'f1_random_score', 'precision_selected', 'precision_random', 'recall_selected', 'recall_random', 'auc_selected', 'auc_random'])

buildings_list = np.unique(washing_machines.index.get_level_values('building')).tolist()

app_subset = washing_machines

selected_power = 0
aggregated_power = 0
days = 0

# Run the experiment

for time_on, time_off in timeslots_list:
    precalculated_probs = {};
    for number_of_selected_households in subset:
        results = pd.DataFrame(columns=['f1_selected_score', 'f1_random_score', 'precision_selected', 'precision_random', 'recall_selected', 'recall_random', 'auc_selected', 'auc_random'])
        print(f'households: {number_of_selected_households}, {time_on}-{time_off}')
        dict_index = {'subset': number_of_selected_households, 'timeslot': f'{time_on}-{time_off}'}
        start_time = time.time()
        for k in range (0,number_of_runs):
            selected_power = 0
            aggregated_power = 0
            days = 0
            end_REFIT = maxdate.replace(hour=pd.Timestamp(time_off).hour, minute =0)
            day = trainset.replace(hour=pd.Timestamp(time_on).hour, minute =0)
            selected = pd.DataFrame(columns=['day', 'p'])
            random = pd.DataFrame(columns=['day'])
            #onlist = pd.DataFrame(columns=['state', 'day', 'selected', 'random'])
            df_x = []
 #           day2 = day + pd.Timedelta(days=1)
#            day3 = day2 + pd.Timedelta(days=1)
            #iterate over days
            while day < end_REFIT:
                print(day)
                thestate = pd.DataFrame(columns=['state', 'day', 'selected', 'random', 'p'])
                p = pd.DataFrame(columns=['p'])
                day_end = day.replace(hour=pd.Timestamp(time_off).hour, minute =0)
                #iterate over buildings
                count = 1
                for i in np.unique(app_subset.index.get_level_values('building')):
                    #calculating probabilities for a certain time slot
                    #the id number of buildings as an integer
                    #saving the probabilities - εδώ διορθώθηκε το i+1 ορίζοντας το index ως i
                    aprob = 0
                    oy = off_yesterday(day, on_times_all[count-1])
                    if (count-1, oy) in precalculated_probs:
                        aprob = precalculated_probs[(count-1, oy)]
                    else:
                        aprob = slot(prob_days[count-1], prob_times[count-1], prob_dur[count-1], day, day_end, oy)
                        precalculated_probs[(count-1, oy)] = aprob
                    p = p.append(pd.DataFrame({'p': aprob}, index=[count]))
                    # έβγαζε κάποιο λάθος η pandas και γι αυτό δεν το περνάω κατευθείαν στο onlist
                    a = (app_subset.loc[pd.IndexSlice[app_subset.index.get_level_values('building') == i, :, day:day_end]]['active'] > 50).any()
                    thestate = thestate.append(pd.DataFrame({'state': a, 'day': day, 'selected': False, 'random': False}, index=[count]))
                    count = count + 1
                #choose random buildings to send a message
                prand = np.random.rand(len(buildings_list))
                # randomlist = sample(buildings_list, k=number_of_selected_households)
                randomlist = list(np.argsort(prand)[::-1][:number_of_selected_households])
                #choose first buildings by sorting based on the probabilities
                thelist = p.sort_values(by=['p'], ascending=False).head(number_of_selected_households)
                #saving selected and random buildings for each day
                thestate['selected'].mask(thestate.index.isin(thelist.index), True, inplace=True)
                thestate['random'].mask(thestate.index.isin(randomlist), True, inplace=True)
                thestate['p'] = p
                thestate['prand'] = prand
                #saving  data for each day/building
                #  = pd.concat([random, pd.DataFrame({'day': day}, index=randomlist)])
                #selected = pd.concat([selected, pd.DataFrame({'day': day, 'p': thelist['p']}, index=thelist.index)])
                
                # I have selected random list and thelist.index
                #selected_power = selected_power + washing_machines_common_ts.loc[pd.IndexSlice[thelist.index,:,str(day.date())],:].groupby('time').sum()
                #aggregated_power = aggregated_power + aggregates_common_ts.loc[pd.IndexSlice[:,str(day.date())],:].groupby('time').sum()
                #days = days + 1
                df_x.append(thestate)
                #onlist = pd.concat([onlist, thestate])
                day += pd.Timedelta(days=1)
            onlist = pd.concat(df_x, axis=0)
            #print(selected_power/days)
            #print(aggregated_power/days)
            states = np.array(onlist['state']).tolist()
            selected_list = np.array(onlist['selected']).tolist()
            random_list = np.array(onlist['random']).tolist()
            probs = np.array(onlist['p']).tolist()
            randprobs = np.array(onlist['prand']).tolist()
            f1_selected_score = f1_score(states, selected_list, average='binary')
            #print("F1 selected score ", f1_selected_score)
            f1_random_score = f1_score(states, random_list, average='binary')
            #print("F1 random score ", f1_random_score )
            auc_selected_score = roc_auc_score(states, probs)
            #print("AUC selected score ", auc_selected_score )
            auc_random_score = roc_auc_score(states, randprobs)
            #print("AUC random score ", auc_random_score )
            precision_selected = precision_score(states, selected_list, average='binary')
            #print("Precision score selected", precision_selected)
            precision_random = precision_score(states, random_list, average='binary')
            #print("Precision score random", precision_random)
            recall_selected = recall_score(states, selected_list, average='binary')
            #print("Recall score selected",recall_selected)
            recall_random = recall_score(states, random_list, average='binary')
            #print("Recall score random", recall_random)
            #balanced_selected = balanced_accuracy_score(states, selected_list)
            #print("Balanced accuracy score selected ", balanced_selected)
            #balanced_random = balanced_accuracy_score(states, random_list)
            #print("Balanced accuracy score random ", balanced_random)
            #accuracy_selected = accuracy_score(states, selected_list)
            #print("Accuracy score selected ", accuracy_selected)
            #accuracy_random = accuracy_score(states, random_list)
            #print("Accuracy score random ", accuracy_random)
            results = results.append(
                pd.DataFrame({
                    'f1_selected_score':f1_selected_score, 
                    'f1_random_score':f1_random_score, 
                    'precision_selected': precision_selected,
                    'precision_random': precision_random,
                    'recall_selected': recall_selected,
                    'recall_random': recall_random,
                    'auc_selected':auc_selected_score, 
                    'auc_random': auc_random_score}, index=[k])
            )
        # get the average of recall for random and selected
        # put in in the result list
#        print(results.mean())
        dict_results = results.mean().to_dict()
        final_grid_results = final_grid_results.append({**dict_index, **dict_results}, ignore_index=True)
        print('minutes: ',(time.time() - start_time)/60)

households: 2, 17:00:00-19:00:00
2014-11-21 17:00:00
2014-11-22 17:00:00
2014-11-23 17:00:00
2014-11-24 17:00:00
2014-11-25 17:00:00
2014-11-26 17:00:00
2014-11-27 17:00:00
2014-11-28 17:00:00
2014-11-29 17:00:00
2014-11-30 17:00:00
2014-12-01 17:00:00
2014-12-02 17:00:00
2014-12-03 17:00:00
2014-12-04 17:00:00
2014-12-05 17:00:00
2014-12-06 17:00:00
2014-12-07 17:00:00
2014-12-08 17:00:00
2014-12-09 17:00:00
2014-12-10 17:00:00
2014-12-11 17:00:00
2014-12-12 17:00:00
2014-12-13 17:00:00
2014-12-14 17:00:00
2014-12-15 17:00:00
2014-12-16 17:00:00
2014-12-17 17:00:00
2014-12-18 17:00:00
2014-12-19 17:00:00
2014-12-20 17:00:00
2014-12-21 17:00:00
2014-12-22 17:00:00
2014-12-23 17:00:00
2014-12-24 17:00:00
2014-12-25 17:00:00
2014-12-26 17:00:00
2014-12-27 17:00:00
2014-12-28 17:00:00
2014-12-29 17:00:00
2014-12-30 17:00:00
2014-12-31 17:00:00
2015-01-01 17:00:00
2015-01-02 17:00:00
2015-01-03 17:00:00
2015-01-04 17:00:00
2015-01-05 17:00:00
2015-01-06 17:00:00
2015-01-07 17:00:00
2015-01

2015-01-26 17:00:00
2015-01-27 17:00:00
2015-01-28 17:00:00
2015-01-29 17:00:00
2015-01-30 17:00:00
2015-01-31 17:00:00
2015-02-01 17:00:00
2015-02-02 17:00:00
2015-02-03 17:00:00
2015-02-04 17:00:00
2015-02-05 17:00:00
2015-02-06 17:00:00
2015-02-07 17:00:00
2015-02-08 17:00:00
2015-02-09 17:00:00
2015-02-10 17:00:00
2015-02-11 17:00:00
2015-02-12 17:00:00
2015-02-13 17:00:00
2015-02-14 17:00:00
2015-02-15 17:00:00
2015-02-16 17:00:00
2015-02-17 17:00:00
2015-02-18 17:00:00
2015-02-19 17:00:00
2015-02-20 17:00:00
2015-02-21 17:00:00
2015-02-22 17:00:00
2015-02-23 17:00:00
2015-02-24 17:00:00
2015-02-25 17:00:00
2015-02-26 17:00:00
2015-02-27 17:00:00
2015-02-28 17:00:00
2015-03-01 17:00:00
2015-03-02 17:00:00
2015-03-03 17:00:00
2015-03-04 17:00:00
2015-03-05 17:00:00
2015-03-06 17:00:00
2015-03-07 17:00:00
2015-03-08 17:00:00
2015-03-09 17:00:00
2015-03-10 17:00:00
2015-03-11 17:00:00
2015-03-12 17:00:00
2015-03-13 17:00:00
2015-03-14 17:00:00
2015-03-15 17:00:00
2015-03-16 17:00:00


2015-04-04 17:00:00
2015-04-05 17:00:00
2015-04-06 17:00:00
2015-04-07 17:00:00
2015-04-08 17:00:00
2015-04-09 17:00:00
2015-04-10 17:00:00
2015-04-11 17:00:00
2015-04-12 17:00:00
2015-04-13 17:00:00
2015-04-14 17:00:00
2015-04-15 17:00:00
2015-04-16 17:00:00
2015-04-17 17:00:00
2015-04-18 17:00:00
2015-04-19 17:00:00
2015-04-20 17:00:00
2015-04-21 17:00:00
2015-04-22 17:00:00
2015-04-23 17:00:00
2015-04-24 17:00:00
2015-04-25 17:00:00
2015-04-26 17:00:00
2015-04-27 17:00:00
2015-04-28 17:00:00
2015-04-29 17:00:00
2015-04-30 17:00:00
2015-05-01 17:00:00
2015-05-02 17:00:00
2015-05-03 17:00:00
2015-05-04 17:00:00
2015-05-05 17:00:00
2015-05-06 17:00:00
2015-05-07 17:00:00
2015-05-08 17:00:00
2015-05-09 17:00:00
2015-05-10 17:00:00
minutes:  11.348760716120402
households: 4, 17:00:00-19:00:00
2014-11-21 17:00:00
2014-11-22 17:00:00
2014-11-23 17:00:00
2014-11-24 17:00:00
2014-11-25 17:00:00
2014-11-26 17:00:00
2014-11-27 17:00:00
2014-11-28 17:00:00
2014-11-29 17:00:00
2014-11-30 17:00:0

2014-12-19 17:00:00
2014-12-20 17:00:00
2014-12-21 17:00:00
2014-12-22 17:00:00
2014-12-23 17:00:00
2014-12-24 17:00:00
2014-12-25 17:00:00
2014-12-26 17:00:00
2014-12-27 17:00:00
2014-12-28 17:00:00
2014-12-29 17:00:00
2014-12-30 17:00:00
2014-12-31 17:00:00
2015-01-01 17:00:00
2015-01-02 17:00:00
2015-01-03 17:00:00
2015-01-04 17:00:00
2015-01-05 17:00:00
2015-01-06 17:00:00
2015-01-07 17:00:00
2015-01-08 17:00:00
2015-01-09 17:00:00
2015-01-10 17:00:00
2015-01-11 17:00:00
2015-01-12 17:00:00
2015-01-13 17:00:00
2015-01-14 17:00:00
2015-01-15 17:00:00
2015-01-16 17:00:00
2015-01-17 17:00:00
2015-01-18 17:00:00
2015-01-19 17:00:00
2015-01-20 17:00:00
2015-01-21 17:00:00
2015-01-22 17:00:00
2015-01-23 17:00:00
2015-01-24 17:00:00
2015-01-25 17:00:00
2015-01-26 17:00:00
2015-01-27 17:00:00
2015-01-28 17:00:00
2015-01-29 17:00:00
2015-01-30 17:00:00
2015-01-31 17:00:00
2015-02-01 17:00:00
2015-02-02 17:00:00
2015-02-03 17:00:00
2015-02-04 17:00:00
2015-02-05 17:00:00
2015-02-06 17:00:00


2015-02-25 17:00:00
2015-02-26 17:00:00
2015-02-27 17:00:00
2015-02-28 17:00:00
2015-03-01 17:00:00
2015-03-02 17:00:00
2015-03-03 17:00:00
2015-03-04 17:00:00
2015-03-05 17:00:00
2015-03-06 17:00:00
2015-03-07 17:00:00
2015-03-08 17:00:00
2015-03-09 17:00:00
2015-03-10 17:00:00
2015-03-11 17:00:00
2015-03-12 17:00:00
2015-03-13 17:00:00
2015-03-14 17:00:00
2015-03-15 17:00:00
2015-03-16 17:00:00
2015-03-17 17:00:00
2015-03-18 17:00:00
2015-03-19 17:00:00
2015-03-20 17:00:00
2015-03-21 17:00:00
2015-03-22 17:00:00
2015-03-23 17:00:00
2015-03-24 17:00:00
2015-03-25 17:00:00
2015-03-26 17:00:00
2015-03-27 17:00:00
2015-03-28 17:00:00
2015-03-29 17:00:00
2015-03-30 17:00:00
2015-03-31 17:00:00
2015-04-01 17:00:00
2015-04-02 17:00:00
2015-04-03 17:00:00
2015-04-04 17:00:00
2015-04-05 17:00:00
2015-04-06 17:00:00
2015-04-07 17:00:00
2015-04-08 17:00:00
2015-04-09 17:00:00
2015-04-10 17:00:00
2015-04-11 17:00:00
2015-04-12 17:00:00
2015-04-13 17:00:00
2015-04-14 17:00:00
2015-04-15 17:00:00


...

2015-01-18 17:00:00
2015-01-19 17:00:00
2015-01-20 17:00:00
2015-01-21 17:00:00
2015-01-22 17:00:00
2015-01-23 17:00:00
2015-01-24 17:00:00
2015-01-25 17:00:00
2015-01-26 17:00:00
2015-01-27 17:00:00
2015-01-28 17:00:00
2015-01-29 17:00:00
2015-01-30 17:00:00
2015-01-31 17:00:00
2015-02-01 17:00:00
2015-02-02 17:00:00
2015-02-03 17:00:00
2015-02-04 17:00:00
2015-02-05 17:00:00
2015-02-06 17:00:00
2015-02-07 17:00:00
2015-02-08 17:00:00
2015-02-09 17:00:00
2015-02-10 17:00:00
2015-02-11 17:00:00
2015-02-12 17:00:00
2015-02-13 17:00:00
2015-02-14 17:00:00
2015-02-15 17:00:00
2015-02-16 17:00:00
2015-02-17 17:00:00
2015-02-18 17:00:00
2015-02-19 17:00:00
2015-02-20 17:00:00
2015-02-21 17:00:00
2015-02-22 17:00:00
2015-02-23 17:00:00
2015-02-24 17:00:00
2015-02-25 17:00:00
2015-02-26 17:00:00
2015-02-27 17:00:00
2015-02-28 17:00:00
2015-03-01 17:00:00
2015-03-02 17:00:00
2015-03-03 17:00:00
2015-03-04 17:00:00
2015-03-05 17:00:00
2015-03-06 17:00:00
2015-03-07 17:00:00
2015-03-08 17:00:00


2015-03-24 17:00:00
2015-03-25 17:00:00
2015-03-26 17:00:00
2015-03-27 17:00:00
2015-03-28 17:00:00
2015-03-29 17:00:00
2015-03-30 17:00:00
2015-03-31 17:00:00
2015-04-01 17:00:00
2015-04-02 17:00:00
2015-04-03 17:00:00
2015-04-04 17:00:00
2015-04-05 17:00:00
2015-04-06 17:00:00
2015-04-07 17:00:00
2015-04-08 17:00:00
2015-04-09 17:00:00
2015-04-10 17:00:00
2015-04-11 17:00:00
2015-04-12 17:00:00
2015-04-13 17:00:00
2015-04-14 17:00:00
2015-04-15 17:00:00
2015-04-16 17:00:00
2015-04-17 17:00:00
2015-04-18 17:00:00
2015-04-19 17:00:00
2015-04-20 17:00:00
2015-04-21 17:00:00
2015-04-22 17:00:00
2015-04-23 17:00:00
2015-04-24 17:00:00
2015-04-25 17:00:00
2015-04-26 17:00:00
2015-04-27 17:00:00
2015-04-28 17:00:00
2015-04-29 17:00:00
2015-04-30 17:00:00
2015-05-01 17:00:00
2015-05-02 17:00:00
2015-05-03 17:00:00
2015-05-04 17:00:00
2015-05-05 17:00:00
2015-05-06 17:00:00
2015-05-07 17:00:00
2015-05-08 17:00:00
2015-05-09 17:00:00
2015-05-10 17:00:00
2014-11-21 17:00:00
2014-11-22 17:00:00


2014-12-11 17:00:00
2014-12-12 17:00:00
2014-12-13 17:00:00
2014-12-14 17:00:00
2014-12-15 17:00:00
2014-12-16 17:00:00
2014-12-17 17:00:00
2014-12-18 17:00:00
2014-12-19 17:00:00
2014-12-20 17:00:00
2014-12-21 17:00:00
2014-12-22 17:00:00
2014-12-23 17:00:00
2014-12-24 17:00:00
2014-12-25 17:00:00
2014-12-26 17:00:00
2014-12-27 17:00:00
2014-12-28 17:00:00
2014-12-29 17:00:00
2014-12-30 17:00:00
2014-12-31 17:00:00
2015-01-01 17:00:00
2015-01-02 17:00:00
2015-01-03 17:00:00
2015-01-04 17:00:00
2015-01-05 17:00:00
2015-01-06 17:00:00
2015-01-07 17:00:00
2015-01-08 17:00:00
2015-01-09 17:00:00
2015-01-10 17:00:00
2015-01-11 17:00:00
2015-01-12 17:00:00
2015-01-13 17:00:00
2015-01-14 17:00:00
2015-01-15 17:00:00
2015-01-16 17:00:00
2015-01-17 17:00:00
2015-01-18 17:00:00
2015-01-19 17:00:00
2015-01-20 17:00:00
2015-01-21 17:00:00
2015-01-22 17:00:00
2015-01-23 17:00:00
2015-01-24 17:00:00
2015-01-25 17:00:00
2015-01-26 17:00:00
2015-01-27 17:00:00
2015-01-28 17:00:00
2015-01-29 17:00:00


2015-02-14 17:00:00
2015-02-15 17:00:00
2015-02-16 17:00:00
2015-02-17 17:00:00
2015-02-18 17:00:00
2015-02-19 17:00:00
2015-02-20 17:00:00
2015-02-21 17:00:00
2015-02-22 17:00:00
2015-02-23 17:00:00
2015-02-24 17:00:00
2015-02-25 17:00:00
2015-02-26 17:00:00
2015-02-27 17:00:00
2015-02-28 17:00:00
2015-03-01 17:00:00
2015-03-02 17:00:00
2015-03-03 17:00:00
2015-03-04 17:00:00
2015-03-05 17:00:00
2015-03-06 17:00:00
2015-03-07 17:00:00
2015-03-08 17:00:00
2015-03-09 17:00:00
2015-03-10 17:00:00
2015-03-11 17:00:00
2015-03-12 17:00:00
2015-03-13 17:00:00
2015-03-14 17:00:00
2015-03-15 17:00:00
2015-03-16 17:00:00
2015-03-17 17:00:00
2015-03-18 17:00:00
2015-03-19 17:00:00
2015-03-20 17:00:00
2015-03-21 17:00:00
2015-03-22 17:00:00
2015-03-23 17:00:00
2015-03-24 17:00:00
2015-03-25 17:00:00
2015-03-26 17:00:00
2015-03-27 17:00:00
2015-03-28 17:00:00
2015-03-29 17:00:00
2015-03-30 17:00:00
2015-03-31 17:00:00
2015-04-01 17:00:00
2015-04-02 17:00:00
2015-04-03 17:00:00
2015-04-04 17:00:00


2015-04-23 17:00:00
2015-04-24 17:00:00
2015-04-25 17:00:00
2015-04-26 17:00:00
2015-04-27 17:00:00
2015-04-28 17:00:00
2015-04-29 17:00:00
2015-04-30 17:00:00
2015-05-01 17:00:00
2015-05-02 17:00:00
2015-05-03 17:00:00
2015-05-04 17:00:00
2015-05-05 17:00:00
2015-05-06 17:00:00
2015-05-07 17:00:00
2015-05-08 17:00:00
2015-05-09 17:00:00
2015-05-10 17:00:00
2014-11-21 17:00:00
2014-11-22 17:00:00
2014-11-23 17:00:00
2014-11-24 17:00:00
2014-11-25 17:00:00
2014-11-26 17:00:00
2014-11-27 17:00:00
2014-11-28 17:00:00
2014-11-29 17:00:00
2014-11-30 17:00:00
2014-12-01 17:00:00
2014-12-02 17:00:00
2014-12-03 17:00:00
2014-12-04 17:00:00
2014-12-05 17:00:00
2014-12-06 17:00:00
2014-12-07 17:00:00
2014-12-08 17:00:00
2014-12-09 17:00:00
2014-12-10 17:00:00
2014-12-11 17:00:00
2014-12-12 17:00:00
2014-12-13 17:00:00
2014-12-14 17:00:00
2014-12-15 17:00:00
2014-12-16 17:00:00
2014-12-17 17:00:00
2014-12-18 17:00:00
2014-12-19 17:00:00
2014-12-20 17:00:00
2014-12-21 17:00:00
2014-12-22 17:00:00


In [127]:
final_grid_results

,subset,timeslot,f1_selected_score,f1_random_score,precision_selected,precision_random,recall_selected,recall_random,auc_selected,auc_random
0,2,17:00:00-19:00:00,0.274725,0.107793,0.292398,0.118356,0.259067,0.098964,0.724653,0.500743
1,4,17:00:00-19:00:00,0.310280,0.153088,0.242690,0.122092,0.430052,0.205181,0.724653,0.489911
2,6,17:00:00-19:00:00,0.315864,0.162750,0.217349,0.113572,0.577720,0.287047,0.724653,0.508414
3,8,17:00:00-19:00:00,0.295325,0.168991,0.189327,0.109552,0.670984,0.369430,0.724653,0.498346
4,10,17:00:00-19:00:00,0.287214,0.186826,0.176023,0.115610,0.779793,0.486528,0.724653,0.502636


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

from scipy import interp
from sklearn.metrics import roc_auc_score

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(states, randprobs)
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure()
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
d = { 'real': states, 'probs': randprobs }
roc_df = pd.DataFrame(d)
roc_df.to_csv('demo.csv')

In [ ]:
b = [1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19,20,21]
list(np.argsort(np.random.rand(len(b)))[::-1][:2])

In [ ]:
final_grid_results

In [ ]:
sum(states)

In [ ]:
aggregates_common_ts.loc[pd.IndexSlice[8, '2015-05-09', :],:].groupby(['time']).sum()

In [ ]:
283.014815 + 1699.649254 + 494.225564

In [ ]:
days

In [ ]:
selected_power/170

In [ ]:
aggregated_power/170

In [ ]:
maxdate-trainset

In [ ]:
aggregates.loc[idx[sample(a, k=2),'2014-06-03 17:30:00'],'active'].sum()

In [ ]:
washing_machines

In [ ]:
washing_machines_common = washing_machines.loc[pd.IndexSlice[:, :, trainset:maxdate], :]
mask = washing_machines_common.index.get_level_values('Unix').indexer_between_time('17:00', '19:00')
washing_machines_common_ts = washing_machines_common.iloc[mask]
washing_machines_common_ts.index = pd.MultiIndex.from_arrays([
    washing_machines_common_ts.index.get_level_values('building'), 
    washing_machines_common_ts.index.get_level_values('appNo'), 
    washing_machines_common_ts.index.get_level_values('Unix').date, 
    washing_machines_common_ts.index.get_level_values('Unix').time], 
    names=['building','appNo','date','time'])
a = 0
a = a + washing_machines_common_ts.loc[pd.IndexSlice[[1,2],:,'2014-12-22'],:].groupby('time').sum()
a = a + washing_machines_common_ts.loc[pd.IndexSlice[[1,2],:,'2014-12-23'],:].groupby('time').sum()
a